## Part 1 Download Datset and Understand the Format

Import Libraries

In [3]:
import gzip

Read Dataset files

In [4]:
trainDataset = gzip.open('kddcup.data.gz', 'r').readlines()
testDataset = gzip.open('corrected.gz', 'r').readlines()

In [5]:
print(trainDataset[0])
print(testDataset[0])

b'0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.\n'
b'0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.\n'


Maps from categorical data to numerical


In [6]:
attrNames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label']
protocol_type = {'tcp': 0, 'udp': 1, 'icmp': 2}
service = {'aol': 0, 'auth': 1, 'bgp': 2, 'courier': 3, 'csnet_ns': 4, 'ctf': 5, 'daytime': 6, 'discard': 7, 'domain': 8, 'domain_u': 9, 'echo': 10, 'eco_i': 11, 'ecr_i': 12, 'efs': 13, 'exec': 14, 'finger': 15, 'ftp': 16, 'ftp_data': 17, 'gopher': 18, 'harvest': 19, 'hostnames': 20, 'http': 21, 'http_2784': 22, 'http_443': 23, 'http_8001': 24, 'imap4': 25, 'IRC': 26, 'iso_tsap': 27, 'klogin': 28, 'kshell': 29, 'ldap': 30, 'link': 31, 'login': 32, 'mtp': 33, 'name': 34, 'netbios_dgm': 35, 'netbios_ns': 36, 'netbios_ssn': 37, 'netstat': 38, 'nnsp': 39, 'nntp': 40, 'ntp_u': 41, 'other': 42, 'pm_dump': 43, 'pop_2': 44, 'pop_3': 45, 'printer': 46, 'private': 47, 'red_i': 48, 'remote_job': 49, 'rje': 50, 'shell': 51, 'smtp': 52, 'sql_net': 53, 'ssh': 54, 'sunrpc': 55, 'supdup': 56, 'systat': 57, 'telnet': 58, 'tftp_u': 59, 'tim_i': 60, 'time': 61, 'urh_i': 62, 'urp_i': 63, 'uucp': 64, 'uucp_path': 65, 'vmnet': 66, 'whois': 67, 'X11': 68, 'Z39_50': 69, 'icmp': 70}
flag = {'OTH': 0, 'REJ': 1, 'RSTO': 2, 'RSTOS0': 3, 'RSTR': 4, 'S0': 5, 'S1': 6, 'S2': 7, 'S3': 8, 'SF': 9, 'SH': 10}
labels = {'normal': 0, 'back': 1, 'buffer_overflow': 2, 'ftp_write': 3, 'guess_passwd': 4, 'imap': 5, 'ipsweep': 6, 'land': 7, 'loadmodule': 8, 'multihop': 9, 'neptune': 10, 'nmap': 11, 'perl': 12, 'phf': 13, 'pod': 14, 'portsweep': 15, 'rootkit': 16, 'satan': 17, 'smurf': 18, 'spy': 19, 'teardrop': 20, 'warezclient': 21, 'warezmaster': 22, 'snmpgetattack': 23, 'snmpguess': 24, 'httptunnel': 25, 'sendmail': 26, 'named': 27, 'xlock': 28, 'xsnoop': 29, 'worm': 30, 'xterm': 31, 'ps': 32, 'sqlattack': 33, 'udpstorm': 34, 'mailbomb': 35, 'saint': 36, 'apache2': 37, 'mscan': 38, 'processtable': 39,'icmp': 40}

Change the categorical features to numerical

In [7]:
for i in range(len(trainDataset)):
    
    trainDataset[i] = trainDataset[i].decode('utf-8') # convert from bytes to string
    if trainDataset[i].endswith('.\n'): 
        trainDataset[i] = trainDataset[i].replace('.\n', '') # remove '.\n'
    trainDataset[i] = trainDataset[i].strip().split(',') # split by comma
    
    for j in range(len(trainDataset[i])):
        try:
            trainDataset[i][j] = int(trainDataset[i][j]) # convert to int
        except ValueError:
            try:
                trainDataset[i][j] = float(trainDataset[i][j]) # convert to float
            except ValueError: # convert categorical data to numerical data
                if j == 1: 
                    trainDataset[i][j] = protocol_type[trainDataset[i][j]]
                elif j == 2:
                    trainDataset[i][j] = service[trainDataset[i][j]]
                elif j == 3:
                    trainDataset[i][j] = flag[trainDataset[i][j]]
                elif j == 41:
                    trainDataset[i][j] = labels[trainDataset[i][j]]

In [8]:
# convert from bytes to string and change categorical data to numerical data
for i in range(len(testDataset)):
    
    testDataset[i] = testDataset[i].decode('utf-8') # convert from bytes to string
    if testDataset[i].endswith('.\n'):
        testDataset[i] = testDataset[i].replace('.\n', '') # remove '.\n'
    testDataset[i] = testDataset[i].strip().split(',') # split by comma
    
    for j in range(len(testDataset[i])):
        try:
            testDataset[i][j] = int(testDataset[i][j]) # convert to int
        except ValueError:
            try:
                testDataset[i][j] = float(testDataset[i][j]) # convert to float
            except ValueError: # convert categorical data to numerical data
                if j == 1:
                    testDataset[i][j] = protocol_type[testDataset[i][j]]
                elif j == 2:
                    testDataset[i][j] = service[testDataset[i][j]]
                elif j == 3:
                    testDataset[i][j] = flag[testDataset[i][j]]
                elif j == 41:
                    testDataset[i][j] = labels[testDataset[i][j]]

In [12]:
print(len(trainDataset[0]))
print(len(testDataset[0]))

42
42
